In [ ]:
pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 5.3 MB/s eta 0:00:00


In [ ]:
pip install webdriver-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.PbEUkyvez4/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.dsKO2r8avI/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.pYgCZ40HIp/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
# Install chromium browser and driver
!apt-get update
!apt-get install chromium chromium-driver

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:4 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:11 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:13 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Get:14 http://ppa.launchpad.net/deadsnak

In [ ]:
# Create Selenium options
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver


In [ ]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import numpy as np
import time

driver = web_driver()

def property_for_sale_hcm(num_page,keyword):
    keyword = keyword.replace(' ','-')
    #Create job title, company, dates post, skills, salary list
    price = []
    address = []
    lot_area = []
    details = []
    for page in range(1,num_page+1):    
        url = f'https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/2/{keyword}/trang--{page}.html'
        driver.maximize_window()        #For maximizing window
        driver.implicitly_wait(10)      #Gives an implicit wait for 10 seconds
        driver.get(url)
        total_height = int(driver.execute_script('return document.body.scrollHeight'))
        for i in range(1, total_height, 100):
            driver.execute_script('window.scrollTo(0, {});'.format(i))
        #Get price 
        price_element = driver.find_elements(by=By.CSS_SELECTOR,value='[class="ct_price"]')
        price_func = np.vectorize(lambda x:x.text, otypes=[object])
        if len(list(price_func(price_element))) == 20:
          p1 = list(price_func(price_element))
        else:
          pass
        #Get address
        district_element = driver.find_elements(by=By.CSS_SELECTOR,value='[class="ct_dis"]')
        district_func = np.vectorize(lambda x:x.text, otypes=[object])
        if len(list(district_func(district_element))) == 20:
          a1 = list(district_func(district_element))
        else:
          pass
        # Get lot_area
        area_element = driver.find_elements(by=By.CSS_SELECTOR,value='[class="ct_dt"]')
        area_func = np.vectorize(lambda x:x.text, otypes=[object])
        if len(list(area_func(area_element))) == 20:
          l1 = list(area_func(area_element))
        else:
          pass
        # Get details
        detail_element = driver.find_elements(by=By.CSS_SELECTOR,value='[class="characteristics"]')
        detail_func = np.vectorize(lambda x:'-'.join(x.text.split('\n')[:]), otypes=[object])
        if len(list(detail_func(detail_element))) == 20:
          d1 = list(detail_func(detail_element))
          price += p1
          address += a1
          lot_area += l1
          details += d1
        else:
          pass
      
        #Create dataframe combining review information
        df = pd.DataFrame({
            'Address': address,
            'Lot_area': lot_area,
            'Details': details,
            'Price': price
        })
        df.to_csv(f'property_for_sale_{keyword}.csv',index=False)
        time.sleep(3)
        
    return df
df = property_for_sale_hcm(keyword='ho chi minh',num_page=50)

In [ ]:
df

,Address,Lot_area,Details,Price
0,"Đường Lê Văn Lương, Xã Phước Kiển, Huyện Nhà B...",Diện tích: 236 m\n2,8m2 lầu13 phòng ngủ,"Giá: 9,99 tỷ"
1,"Đường Vũ Tùng, Phường 2, Quận Bình Thạnh, Hồ C...",Diện tích: 227 m\n2,6m3 lầu25 phòng ngủ,"Giá: 22,9 tỷ"
2,"Đường Nguyễn Hữu Thọ, Phường Tân Phong, Quận 7...",Diện tích: 100 m\n2,12m6 lầu7 phòng ngủ,Giá: 13 tỷ
3,"Đường Nguyễn Văn Bứa, Xã Xuân Thới Thượng, Huy...",Diện tích: 125 m\n2,8m1 lầu3 phòng ngủchỗ để xe,"Giá: 1,75 tỷ"
4,"Đường Nguyễn Văn Bứa, Xã Xuân Thới Thượng, Huy...",Diện tích: 182 m\n2,5m2 lầu3 phòng ngủchỗ để xe,"Giá: 1,75 tỷ"
...,...,...,...,...
855,"Đường Thống Nhất, Phường 11, Quận Gò Vấp, Hồ C...",Diện tích: 120 m\n2,25m2 lầu4 phòng ngủ,"Giá: 14,5 tỷ"
856,"Đường Minh Phụng, Phường 9, Quận 6, Hồ Chí Minh",Diện tích: 90 m\n2,7m1 lầu2 phòng ngủchỗ để xe,"Giá: 11,5 tỷ"
857,"Đường Lê Thị Riêng, Phường Thới An, Quận 12, H...",Diện tích: 58 m\n2,5m2 lầu2 phòng ngủ,"Giá: 3,8 tỷ"
858,"Đường Đinh Bộ Lĩnh, Phường 25, Quận Bình Thạnh...",Diện tích: 73 m\n2,20m8 lầu8 phòng ngủ,Giá: 35 tỷ


In [ ]:
# Tải file excel kết quả về gdrive
df.to_excel(r'drive/MyDrive/Do an cuoi ky/50_continue_pages(15)(copied).xlsx', index=False)

In [ ]:
import pandas as pd
df = pd.DataFrame()
for x in rd:
  i = 0
  for y in x:
    if len(y) == 20:
      i += 1
      if i == 4:
        df1 = pd.DataFrame(x)
        df1 = df1.T
        df1.columns = ['Price', 'Address', 'Lot Area', 'Details']
        df = pd.concat([df, df1], axis=0)
        break
    else:
      break

df 

,Price,Address,Lot Area,Details
0,"Giá: 3,73 tỷ","Đường Thoại Ngọc Hầu, Phường Phú Thạnh, Quận T...",Diện tích: 36 m\n2,7m2 lầu2 phòng ngủ
1,Giá: 12 tỷ,"Đường Phan Chu Trinh, Phường Tân Thành, Quận T...",Diện tích: 78 m\n2,8m3 lầu3 phòng ngủchỗ để xe
2,"Giá: 2,5 tỷ","Tỉnh Lộ 7, Xã Trung Lập Hạ, Huyện Củ Chi, Hồ C...",Diện tích: 2.030 m\n2,5m4 phòng ngủchỗ để xe
3,Giá: 33 tỷ,"Đường Lê Quang Định, Phường 7, Quận Bình Thạnh...",Diện tích: 150 m\n2,30m3 lầu5 phòng ngủ
4,"Giá: 1,4 tỷ","Đường Tân Hiệp 32, Xã Tân Hiệp, Huyện Hóc Môn,...",Diện tích: 440 m\n2,10m
...,...,...,...,...
15,"Giá: 34,5 tỷ","Đường Điện Biên Phủ, Phường 17, Quận Bình Thạn...",Diện tích: 95 m\n2,20m5 lầu5 phòng ngủchỗ để xe
16,"Giá: 4,98 tỷ","Đường Huỳnh Tấn Phát, Phường Tân Thuận Đông, Q...",Diện tích: 44 m\n2,3m1 lầu2 phòng ngủ
17,"Giá: 15,5 tỷ","Đường Gò Dầu, Phường Tân Sơn Nhì, Quận Tân Phú...",Diện tích: 228 m\n2,10mchỗ để xe
18,Giá: 54 tỷ,"Đường Trần Não, Phường An Khánh, Thành phố Thủ...",Diện tích: 132 m\n2,30m4 lầu4 phòng ngủ


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Preprocessing data**

In [ ]:
# Create street, ward, district, street_width, floor, bed_room, garage

street_name = []
district_name = []

for i in df['Address']:
  if 'Thủ Đức' in i:
    if 'Đường số' in i:
      l = i.split(', ')
      street = [', '.join(l[0:2])]
      street_name.append(street)
    else:
      l= i.split(', ')
      street = l[0]
      street_name.append(street)
    district_name.append(['Thủ Đức'])
  else:
    l= i.split(', ')
    street = l[0]
    street_name.append(street)
    district_name.append(l[-2])

# Lot Area
lot_area = df['Lot_area'].split(' ')[2]

# Details
street_width = []
bed_room = []
garage = []
floor = []
for i in df.Details:
  if 'chỗ để xe' in i:
    garage.append(1)
    i1=i.split('chỗ để xe')[0]
    if 'phòng ngủ' in i1:
      i2 = i1.split(' ')[-3]
      bed = i2.split('lầu')[-1]
      bed_room.append(bed)
    else:
      bed_room.append(None)
  else:
    garage.append(0)
    if 'phòng ngủ' in i:
      i1 = i.split(' ')[-3]
      bed = i1.split('lầu')[-1]
      bed_room.append(bed)
    else:
      bed_room.append(None)

for i in df.Details:
  if 'm' in i:
    i1 = i.split('m')
    width = i1[0]
    street_width.append(width)
    if 'lầu' in i:
      floor_num = i1.split(' ')[0]
      floor.append(floor_num)
    else:
      floor.append(None)
  else:
    street_width.append(None)
    if 'lầu' in i:
      floor_num = i.split(' ')[0]
      floor.append(floor_num)
    else:
      floor.append(None)


# Price 
price = df['Price'].split(' ')[1]



In [ ]:
street_name

In [ ]:
district_name

In [ ]:
lot_area

In [ ]:
price

In [ ]:
street_width

In [ ]:
bed_room

In [ ]:
garage

In [ ]:
floor